In [ ]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd

### Project was done on AWS so the data had to be moved from the s3 bucket

In [ ]:
!aws s3 cp s3://michaelmanoukianlandmarks/landmark_images/test/ landmark_images/test/ --recursive #stores them at landmark_images/test/

In [ ]:
!aws s3 cp s3://michaelmanoukianlandmarks/landmark_images/train/ landmark_images/train/ --recursive #stores them at landmark_images/train/

### Displays an image to make sure everything is working

In [ ]:
image = imread('landmark_images/train/16.Eiffel_Tower/640a59e7290ab2c2.jpg')
imshow(image)

### Creates data_loaders and defines the classes

In [ ]:
#sets parameters
num_workers = 0
batch_size = 1
valid_size = 0.2

test_path = r'landmark_images/test/'
train_path = r'landmark_images/train/'

#converts data to normalized vector of the appropriate size
transform = transforms.Compose([transforms.Resize((400,400)),
                                transforms.ToTensor()])

#all the data
test_data = datasets.ImageFolder(root=test_path, transform=transform)
train_data = datasets.ImageFolder(root=train_path, transform=transform)

#splits train and validation data 
num_total = len(train_data)
indices = list(range(num_total))
np.random.shuffle(indices)
split = int(np.floor(num_total * valid_size))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                          sampler= train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                          sampler= valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          num_workers=num_workers)

# defines classes
classes_pre = []
for root, dirs, files in os.walk(test_path, topdown=False):
    for name in dirs:
        x = name.split('.')
        classes_pre.append(x)

# rearanges classes
n = 0
classes = []
for x in range(len(classes_pre)):
    for i in classes_pre:
        if int(i[0]) == n:
            classes.append(i[1])
            n += 1
print(classes)

### Displays images with labels to make sure the data is ready

In [ ]:
dataiter = iter(valid_loader)
images, labels = dataiter.next()
images = images.numpy()

fig = plt.figure(figsize=(25, 5))
#.permute(0,2,3,1)
for idx in np.arange(4):
    ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
    imshow(images[idx].T)
    ax.set_title(classes[labels[idx]])

### Defines CNN

In [ ]:
from torch import nn, optim

#actual neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 11, padding=5)
        self.conv2 = nn.Conv2d(64, 64, 11, padding=5)
        self.conv3 = nn.Conv2d(64, 128, 9, padding=4)
        self.conv4 = nn.Conv2d(128, 128, 9, padding=4)
        self.conv5 = nn.Conv2d(128, 256, 7, padding=3)
        self.conv6 = nn.Conv2d(256, 256, 7, padding=3)
        self.conv7 = nn.Conv2d(256, 256, 7, padding=3)
        self.conv8 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv9 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv11 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        
        self.pool2 = nn.MaxPool2d(2, 2)
        self.pool5 = nn.MaxPool2d(5, 5)
        
        self.fc1 = nn.Linear(512*4*4, 3000)
        self.fc2 = nn.Linear(3000, 100)
        self.fc3 = nn.Linear(1000, 250)
        self.fc4 = nn.Linear(250, 50)
        
        self.dropout = nn.Dropout(p=0.2)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                n = m.in_features
                y = 1.0/np.sqrt(n)
                m.weight.data.normal_(0,y)
                m.bias.data.fill_(0)
        
    def forward(self, x):
        x0 = x.shape[0]
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool5(F.relu(self.conv7(x)))
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = self.pool5(x)
        
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = x.view(x0, -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        
        return x

model = Net()
print(model)

### Trains model

In [ ]:
import gc

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
epochs = 10

#clears cache
torch.cuda.empty_cache()
#sets processing to gpu
model.cuda()
#validation loss min for saving model
valid_loss_min = 10000


for e in range(epochs):
    #tracking loss
    train_loss = 0.0
    valid_loss = 0.0
    
    #sets to training mode
    model.train()
    for images, labels in train_loader:
        #clears optimizer and moves data to cuda
        optimizer.zero_grad()
        images, labels = images.cuda(), labels.cuda()
        
        #runs through model
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #updates loss
        train_loss += loss.item()*images.size(0)
        
        #clears cache
        del outputs
        gc.collect()
        torch.cuda.empty_cache()
    
    else:
        #sets to validation mode
        model.eval()
        for images, labels in valid_loader:
            #switches to gpu
            images, labels = images.cuda(), labels.cuda()
        
            #runs the validation set
            with torch.no_grad():
                outputs = model(images)
                loss = criterion(outputs, labels)
        
            #updates loss
            valid_loss += loss.item()*images.size(0)
            
            #clears cache
            del outputs
            gc.collect()
            torch.cuda.empty_cache()
            
    #updates overall losses
    training_loss = train_loss/len(train_loader.dataset)
    validation_loss = valid_loss/len(valid_loader.dataset)
    
    #prints progress
    print("Epoch: {} \tTraining Loss: {:.4f} \tValidation Loss: {:.4f}".format(
        e+1, training_loss, validation_loss))
    
    #updates model
    if valid_loss <= valid_loss_min:
        torch.save(model.state_dict(), 'landmark_detection_model.pt')
        valid_loss_min = valid_loss
        print('Saving Model...')
    
print('Done!')

### Testing how well the network performed
load images with the predication and the correct label printed above the images. 5 images/predication/labels per class 

In [ ]:
model.eval()

for images, labels in test_loader:
    images, labels = images.cuda(), labels.cuda()

    with torch.no_grad():
        output = model(images)
        loss = criterion(output, labels)
        
     _ , guess_tensor = torch.topk(output, 1)
    guess = np.squeeze(guess_tensor.cpu().numpy())
    
    images = images.cpu().numpy()
    fig = plt.figure(figsize=(25, 5))
    for idx in np.arange(5):
        ax = fig.add_subplot(2, int(20/2), idx+1, xticks=[], yticks=[])
        plt.imshow(np.transpose(images[idx]))
        ax.set_title(str(classes[guess[idx]])+"\n"+str(classes[labels[idx]]))